In [1]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.


In [2]:
# @title GPU details
!nvidia-smi

Sun Dec 26 07:19:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# @title Dependancies and prerequisites
%cd /content/
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113  torchtext  -f https://download.pytorch.org/whl/cu113/torch_stable.html
#!git clone https://github.com/openai/CLIP 
!pip install git+https://github.com/openai/CLIP 
!pip install taming-transformers
!pip install torch_optimizer
#!pip install  git+https://github.com/CompVis/taming-transformers.git
!pip install ftfy regex tqdm omegaconf pytorch-lightning
!pip install kornia
!pip install imageio-ffmpeg   
!pip install einops          
!mkdir -p steps
!git clone https://github.com/nerdyrodent/VQGAN-CLIP.git
%cd VQGAN-CLIP
!git clone https://github.com/openai/CLIP 
!git clone https://github.com/CompVis/taming-transformers.git 
#!pip install torch==1.9.0
#!pip install -U torchtext==0.10.0
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install -r requirements.txt
!python -m bash_kernel.install
!apt install ffmpeg imagemagick bc jq
%cd /content/
!git clone https://github.com/austinhquinn/VQGAN-CLIP.git audioReactive-VQGAN-CLIP 
%cd audioReactive-VQGAN-CLIP 

/content
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 2.4 MB/s eta 0:06:54tcmalloc: large alloc 1147494400 bytes == 0x55b50d2e2000 @  0x7fad470ed615 0x55b4d2f524cc 0x55b4d303247a 0x55b4d2f552ed 0x55b4d3046e1d 0x55b4d2fc8e99 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc8d00 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc5737 0x55b4d3047c66 0x55b4d2fc4daf 0x55b4d3047c66 0x55b4d2fc4daf 0x55b4d3047c66 0x55b4d2fc4daf 0x55b4d2f57039 0x55b4d2f9a409 0x55b4d2f55c52 0x55b4d2fc8c25 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc5737 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc4915 0x55b4d2f56afa 0x55b4d2fc4c0d 0x55b4d2fc39ee
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:22tcmalloc: large alloc 1434370048 bytes == 0x55b551938000 @  0x7fad470ed615 0x55b4d2f524cc 0x55b4d303247a 0x55b4d2f552ed 0x55b4d3046e1d 0x55b4d2fc8e99 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc8d00 0x55b4d2fc39ee 0x55b4d2f56bda 0x55b4d2fc5737 0x55b4d3047c

In [4]:
#@title Selection of models to download
#@markdown By default, the notebook uses imagenet_16384
!mkdir -p /content/VQGAN-CLIP/checkpoints/
%cd /content/VQGAN-CLIP/checkpoints/
imagenet_1024 = False #@param {type:"boolean"}
imagenet_16384 = True #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = False #@param {type:"boolean"}
wikiart_1024 = False #@param {type:"boolean"}
wikiart_16384 = False #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}
openimages_8192 = False #@param {type:"boolean"}

if imagenet_1024:
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.yaml' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.ckpt'  #ImageNet 1024
if imagenet_16384:
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384

if openimages_8192:
  !curl -L -o vqgan_openimages_f16_8192.yaml -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
  !curl -L -o vqgan_openimages_f16_8192.ckpt -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384

if coco:
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
if faceshq:
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
if wikiart_1024: 
  !curl -L -o wikiart_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart.yaml' #WikiArt 1024
  !curl -L -o wikiart_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart.ckpt' #WikiArt 1024
if wikiart_16384: 
  !curl -L -o wikiart_16384.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.yaml' #WikiArt 16384
  !curl -L -o wikiart_16384.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.ckpt' #WikiArt 16384
if sflckr:
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR

%cd /content/VQGAN-CLIP/

/content/VQGAN-CLIP/checkpoints
** Resuming transfer from byte position 692
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
** Resuming transfer from byte position 980092370
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
/content/VQGAN-CLIP


In [41]:
#@title Clean up previous runs and reset dirs and copy demo files
copyDemoFiles=True #@param {type:"boolean"}
!rm -rf /content/steps/* /content/input/* /content/output/*
!mkdir -p /content/input/
if copyDemoFiles:
    %shell cp -f /content/audioReactive-VQGAN-CLIP/input/* /content/input/ 
    %shell cp /content/input/Ssc375.mp3 /content/input/init.mp3

In [42]:
#@title Exec to get init image from polyhaven.com and save it to /content/input/init.png
%cd /content/
!mkdir -p ./input/ ./output/ ./steps/

# convert myexr.exr -colorspace RGB -colorspace sRGB mynonlineartiff.tiff
import os
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import requests
from IPython.display import display 
import shutil
from urllib.parse import urlparse

def getCategories():
    #value=%shell curl -s "https://api.polyhaven.com/categories/hdris"|jq -r ' keys | unique'
    response = requests.get("https://api.polyhaven.com/categories/hdris")
    json_data = json.loads(response.text)
    return json_data


def getAssets(c):
    #value=%shell curl -s "https://api.polyhaven.com/categories/hdris"|jq -r ' keys | unique'
    response = requests.get("https://api.polyhaven.com/assets?t=hdris&c="+c)
    json_data = json.loads(response.text)
    return json_data

def getAssetFiles(c):
    response = requests.get("https://api.polyhaven.com/files/"+c)
    json_data = json.loads(response.text)
    if 'hdri' in json_data:
        returndict={}
        for k in json_data["hdri"].keys():
            #print(json_data["hdri"][k])
            for k2 in json_data["hdri"][k]:
                #print(k2)
                returndict[k+"-"+k2]=json_data["hdri"][k][k2]["url"]
    else:
        returndict= {"no-files":"boo"}
    return returndict

def dlAssetFile(c):
    return c
 

!mkdir -p /content/in/
#!curl -s https://dl.polyhaven.org/file/ph-assets/HDRIs/exr/4k/sunset_jhbcentral_4k.exr -O /content/in/sunset_jhbcentral_4k.exr
#!convert /content/in/sunset_jhbcentral_4k.exr -colorspace RGB -colorspace sRGB /content/in/sunset_jhbcentral_4k.jpg
cates=getCategories()
subcates={}
assetFiles={}
dlassetFiles={}
cate=widgets.Select(
    options=cates, 
    # rows=10,
    description='Category:',
    disabled=False
) 

subcate=widgets.Select(
    options=subcates, 
    # rows=10,
    description='Asset Name:',
    disabled=False
) 
assetFile=widgets.Select(
    options=assetFiles, 
    # rows=10,
    description='Asset Resolution:',
    disabled=False
) 
dlassetFile=widgets.Text(
    options=assetFiles, 
    # rows=10,
    description='Asset URL:',
    disabled=False
) 


def update_x_range(*args):
    subcate.options = getAssets(cate.label)
cate.observe(update_x_range, 'value')

def update_af_range(*args):
    #print(subcate)
    assetFile.options = getAssetFiles(subcate.label)
subcate.observe(update_af_range, 'value')

def update_afinfo_range(*args):
    #print(assetFile.value)
    dlassetFile.value = dlAssetFile(assetFile.value)
assetFile.observe(update_afinfo_range, 'value')

button = widgets.Button(description="Click to download")
def on_button_clicked(b):
    with output:
        print("getting the file"+ dlassetFile.value)
        response = requests.get(dlassetFile.value, stream=True)
        outpath = urlparse(dlassetFile.value).path
        outpath = os.path.basename(outpath)
        with open('/content/input/'+ outpath, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
        del response
        %shell convert {'/content/input/'+ outpath} -colorspace RGB -colorspace sRGB /content/input/init.png
        print("created init image:/content/input/init.png")


button.on_click(on_button_clicked)
output = widgets.Output()
display(cate,subcate,assetFile, dlassetFile,button,output) 

/content


Select(description='Category:', options={'all': 504, 'natural light': 426, 'outdoor': 371, 'urban': 269, 'natu…

Select(description='Asset Name:', options={}, value=None)

Select(description='Asset Resolution:', options={}, value=None)

Text(value='', description='Asset URL:')

Button(description='Click to download', style=ButtonStyle())

Output()

In [60]:
#@title Run Options
import os
import cv2
import math
from decimal import *
TEXT='aliens driving a red 1966 chevy impala lowrider:6| Unreal Engine Ray Tracing:4|Photorealistic:2|text:0|watermark:0|signature:0|logo:0' #@param {type:"string"}
aFILENAME='/content/input/init.mp3' #@param {type:"string"}
FILENAME='/content/input/init.png' #@param {type:"string"}
MAX_ITERATIONS=25 #@param {type:"integer"}
OPTIMISER = 'Adam' #@param ["Adadelta", "Adagrad", "Adam", "Adamax", "Ftrl", "Nadam", "RMSprop", "SGD"]
#@markdown Mount google drive to be able to continue runs
MountGoogleDrive=False #@param {type:"boolean"}
GoogleDrivePath="/content/drive" #@param {type:"string"}
#@markdown Working directory (base path + this value) 
WorkingDirectory="/MyDrive/AIARTWORKDIR/" #@param {type:"string"}
#@markdown Distortion Options
DIST_horizon=False #@param {type:"boolean"} 
DIST_arc=False #@param {type:"boolean"}
DIST_shepards=False #@param {type:"boolean"}
DIST_polar=False #@param {type:"boolean"}
DIST_motion_blur=False #@param {type:"boolean"}
DIST_Zoom=True #@param {type:"boolean"}
DIST_Rotate=True #@param {type:"boolean"}
DIST_virtual_pixel= 'Mirror' #@param ["Mirror", "Tile", "Transparent", "White", "Gray", "Black", "HorizontalTileEdge", "VerticalTileEdge", "CheckerTile", "Random", "Dither"]
#@markdown file used for mp3 audio peak data
sngDataFile='/content/sngData.tmp' #@param {type:"string"}
import os
WorkingDirectoryBase = GoogleDrivePath if MountGoogleDrive==True else '/content/'
os.environ['PROMPT_TEXT'] =str(TEXT)
os.environ['PROMPT_aFILENAME'] =str(aFILENAME)
os.environ['PROMPT_FILENAME'] =str(FILENAME)
os.environ['PROMPT_MAX_ITERATIONS'] =str(MAX_ITERATIONS)
os.environ['PROMPT_OPTIMISER'] =str(OPTIMISER)
os.environ['WorkingDirectoryBase']=str(WorkingDirectoryBase)
os.environ['MountGoogleDrive']=str(MountGoogleDrive)
os.environ['DIST_horizon'] =str(DIST_horizon)
os.environ['DIST_arc'] =str(DIST_arc)
os.environ['DIST_shepards'] =str(DIST_shepards)
os.environ['DIST_polar'] =str(DIST_polar)
os.environ['DIST_motion_blur'] =str(DIST_motion_blur)
os.environ['DIST_Zoom'] =str(DIST_Zoom)
os.environ['DIST_Rotate'] =str(DIST_Rotate)
os.environ['DIST_virtual_pixel'] =str(DIST_virtual_pixel)
os.environ['sngDataFile'] =str(sngDataFile)

if MountGoogleDrive == True:
    from google.colab import drive
    drive.mount(GoogleDrivePath)

os.environ['GoogleDrivePath']=str(MountGoogleDrive)
!export |grep 'GoogleDrivePath'
!export |grep 'WorkingDirectoryBase'
!export |grep 'PROMPT_TEXT'
!export |grep 'PROMPT_aFILENAME'
!export |grep 'PROMPT_FILENAME'
!export |grep 'PROMPT_MAX_ITERATIONS'
!export |grep 'PROMPT_OPTIMISER'

!export |grep 'DIST_horizon'
!export |grep 'DIST_arc'
!export |grep 'DIST_shepards'
!export |grep 'DIST_polar'
!export |grep 'DIST_motion_blur'
!export |grep 'DIST_Zoom'
!export |grep 'DIST_Rotate'
!export |grep 'DIST_virtual_pixel'

!export |grep 'sngDataFile'


# reading res from init image
img2 = cv2.imread(FILENAME)
  
# fetching the dimensions
nXPixels = img2.shape[1]
nYPixels = img2.shape[0]
  
#nXPixels, nYPixels = Image.
nNewPixelResolution=490000 #// 0.49mpx new resolution, limit for 16GB GPU
nOldPixelResolution=nXPixels*nYPixels
conv = Decimal(math.sqrt( Decimal(nNewPixelResolution)/Decimal(nOldPixelResolution)))
width=  round(nXPixels*conv)
height= round(nYPixels*conv)
print("resolution we will use: ("+str(width)+" "+str(height)+")")
print("setting env vars 'PROMPT_W' and 'PROMPT_H' ")
os.environ['PROMPT_W'] =str(width)
os.environ['PROMPT_H'] =str(height)
!export |grep 'PROMPT_W'
!export |grep 'PROMPT_H'



# this next line is ugly. someone send halp. ffmpeg -> json lines
!/usr/bin/ffmpeg -i $PROMPT_aFILENAME -filter_complex ebur128  -f null   -  2>&1 |grep -v 'Summary:' | grep Parsed|sed 's|^\[.*\] ||g' | sed "s|: *|:|g"|sed -r "s|[[:space:]]+| |g" |sed -r "s|LUFS||g" |sed -r "s|LU||g" | sed -r "s|[[:space:]]+|\",\"|g"| sed -r 's|,"$|}|g'|sed -r 's|^|{"|g' |sed -r 's|:|":"|g' > $sngDataFile ;
EPOCHS =   num_lines = sum(1 for line in open(sngDataFile))
# /bin/cat  {sngDataFile}  | /usr/bin/wc -l 
print("setting epochs to: "+str(EPOCHS)) 
os.environ['EPOCHS'] =str(EPOCHS)
!export |grep 'EPOCHS'

declare -x GoogleDrivePath="False"
declare -x WorkingDirectoryBase="/content/"
declare -x PROMPT_TEXT="aliens driving a red 1966 chevy impala lowrider:6| Unreal Engine Ray Tracing:4|Photorealistic:2|text:0|watermark:0|signature:0|logo:0"
declare -x PROMPT_aFILENAME="/content/input/init.mp3"
declare -x PROMPT_FILENAME="/content/input/init.png"
declare -x PROMPT_MAX_ITERATIONS="25"
declare -x PROMPT_OPTIMISER="Adam"
declare -x DIST_horizon="False"
declare -x DIST_arc="False"
declare -x DIST_shepards="False"
declare -x DIST_polar="False"
declare -x DIST_motion_blur="False"
declare -x DIST_Zoom="True"
declare -x DIST_Rotate="True"
declare -x DIST_virtual_pixel="Mirror"
declare -x sngDataFile="/content/sngData.tmp"
resolution we will use: (990 495)
setting env vars 'PROMPT_W' and 'PROMPT_H' 
declare -x PROMPT_W="990"
declare -x PROMPT_H="495"
setting epochs to: 997
declare -x EPOCHS="997"


In [ ]:
#@markdown if you have terminal access, the lines of this file is how many frames will be generated. `wc -l /content/sngData.tmp` you can also click the link to view it in the editor and scroll to the bottom. then check the number of files in the output path.
%%bash
#!/bin/bash
set -x #this turns on command logging
cd /content/VQGAN-CLIP/
# Example audio "Zoom" movie generation
# e.g. ./audio_zoom.sh 'aliens driving a red 1966 chevy impala lowrider:6| Unreal Engine Ray Tracing:4|Photorealistic:2|text:0|watermark:0|signature:0|logo:0' inputs/ssc330B.mp3

TEXT=$PROMPT_TEXT
aFILENAME=$PROMPT_aFILENAME
FILENAME=$PROMPT_FILENAME
#sngDataFile='/content/sngData.tmp'
OUTPATH='/content/output'
GenPath='/content/VQGAN-CLIP/generate.py'
sngLength=`/usr/bin/ffprobe -i ${aFILENAME} -show_entries format=duration -v quiet -of csv="p=0"`

sngLengthRnded=`printf  %.0f "$sngLength"`

echo $?
#POCHS=`sed -n '$=' "$sngDataFile"` ;
#POCHS=`/bin/cat "$sngDataFile" | /usr/bin/wc -l `;

fpsmath="$EPOCHS / $sngLengthRnded";

fps=`echo $fpsmath|bc`;


LR=0.1
OPTIMISER=$PROMPT_OPTIMISER
MAX_ITERATIONS=$PROMPT_MAX_ITERATIONS
maxitersmath="$MAX_ITERATIONS * $fps"
INIT_MAX_ITERATIONS==`echo $maxitersmath|bc` # more iterations for initial /MSE like phase
SEED=`shuf -i 1-9999999999 -n 1` # Keep the same seed each epoch for more deterministic runs

# Extract
FILENAME_NO_EXT=${FILENAME%.*}
FILE_EXTENSION=${FILENAME##*.} 
FILE_BASE=`basename $FILENAME_NO_EXT`
OUT_FILE="${OUTPATH}/${FILE_BASE}"

# Initial run
which python
python $GenPath -p="$TEXT" -opt="$OPTIMISER" -lr=$LR -i=$INIT_MAX_ITERATIONS -se=$INIT_MAX_ITERATIONS --seed=$SEED --size $PROMPT_W $PROMPT_H -o="$FILENAME"
cp -f "$FILENAME" "$OUT_FILE"-00000000."$FILE_EXTENSION"

convert "$FILENAME" -distort SRT 1.01,0 -gravity center "$FILENAME"	# Zoom
convert "$FILENAME" -distort SRT 1 -gravity center "$FILENAME"	# Rotate


# Momentary Loudness measures the loudness of the past 400 Milliseconds.
# Short Term Loudness measures the loudness of the past 3 Seconds.
# Integrated Loudness (Also called Programme Loudness) indicates how loud the programme is on average, and is measured over entire duration of material.
# Loudness Range quantifies, in LU, the statistical distribution of short term loudness within a programme. 
# True Peak indicates accurate measurements of (possible) intersample peaks. 

# Feedback image loop
#EPOCHS=1 #used for testing
for (( i=1; i<=$EPOCHS; i++ ))
do
  #sng_targlufs=`sed -n ${i}p $sngDataFile|jq -r ".TARGET"` #target loudness
  sng_I=`sed -n ${i}p $sngDataFile|jq -r ".I"` #Integrated loudness
  sng_M=`sed -n ${i}p $sngDataFile|jq -r ".M"` #Momentary loudness
  sng_S=`sed -n ${i}p $sngDataFile|jq -r ".S"` #Short-term loudness


  #sng_I_percent=`echo "(100/$sng_targlufs*$sng_I) *0.01" | bc -l` #likely too big/loud
  sng_MofI_percent=`echo "(100/$sng_I*$sng_M) *0.01" | bc -l` #likely too big/loud
  sng_MofS_percent=`echo "(100/$sng_S*$sng_M) *0.01" | bc -l` # current vs last 3 seconds... goldilocks?
  sng_SofI_percent=`echo "(100/$sng_I*$sng_S) *0.01" | bc -l` #likely too big/loud, but 3sec smoothing?
  printf -v sng_lufs_rnd %.0f "$sng_I"
  printf -v sng_M_rnd %.0f "$sng_M"
  printf -v sng_S_rnd %.0f "$sng_S"
  sng_MofS_percent_rnd=`printf  %.0f  "0$sng_MofS_percent"`
  sng_MofS_percent_nondec=`echo "(100/$sng_I*$sng_S)"| bc -l`
  
  padded_count=$(printf "%08d" "$i")  
  echo "$padded_count : $sng_MofS_percent : $sng_SofI_percent"
  python $GenPath -p="$TEXT" -opt="$OPTIMISER" -lr=$LR -i=$MAX_ITERATIONS -se=$MAX_ITERATIONS --seed=$SEED --size $PROMPT_W $PROMPT_H -ii="$FILENAME" -o="$FILENAME"
  cp "$FILENAME" "$OUT_FILE"-"$padded_count"."$FILE_EXTENSION"    
  # https://legacy.imagemagick.org/Usage/misc/#virtual-pixel
  
  # Using Grid Super Sampling https://legacy.imagemagick.org/Usage/distorts/#horizon
  [ $DIST_horizon ] &&  convert "$FILENAME"  -filter point  -set option:distort:scale $sng_MofS_percent_rnd  -virtual-pixel $DIST_virtual_pixel  -distort Perspective '0,0 20,60  90,0 70,63  0,90 5,83  90,90 85,88' -scale $sng_MofS_percent_rnd%    "$FILENAME" 
  
  # https://legacy.imagemagick.org/Usage/distorts/#arc
  #[ $DIST_arc ] && convert "$FILENAME" -virtual-pixel Tile  -distort Arc 360  +repage  "$FILENAME"
  [ $DIST_arc ] && convert "$FILENAME" -virtual-pixel $DIST_virtual_pixel -distort Arc "360 0 $sng_MofS_percent_nondec 0"  +repage  "$FILENAME"
  
  # https://legacy.imagemagick.org/Usage/distorts/#shepards
  [ $DIST_shepards ] && convert "$FILENAME" -virtual-pixel $DIST_virtual_pixel  -define shepards:power=$sng_MofI_percent  -distort Shepards '30,11 20,11  48,29 58,29' "$FILENAME"
  
  # https://legacy.imagemagick.org/Usage/distorts/#polar
  [ $DIST_polar ] && convert "$FILENAME"  -virtual-pixel $DIST_virtual_pixel -distort Barrel "$sng_MofS_percent 0.0 0.0 sng_SofI_percent" "$FILENAME"
  
  # https://legacy.imagemagick.org/Usage/distorts/#depolar
  # https://legacy.imagemagick.org/Usage/blur/#motion-blur
  [ $DIST_motion_blur ] && convert "$FILENAME"   -virtual-pixel $DIST_virtual_pixel  -set option:distort:scale $sng_MofI_percent  -distort DePolar -1  -virtual-pixel Edge   -motion-blur 0x28-90 -virtual-pixel HorizontalTile -background Black  -set option:distort:scale .25 -distort Polar -1  "$FILENAME"
  [ $DIST_Zoom ] && convert "$FILENAME"  -virtual-pixel $DIST_virtual_pixel -distort SRT $sng_SofI_percent,0 -gravity center "$FILENAME" # Zoom
  [ $DIST_Rotate ] && convert "$FILENAME"  -virtual-pixel $DIST_virtual_pixel -distort SRT $sng_MofS_percent -gravity center "$FILENAME"	# Rotate
done

# Make video - Nvidia GPU expected
ffmpeg -y -r $fps -i "$OUT_FILE"-%08d."$FILE_EXTENSION" -b:v 8M -c:v h264_nvenc -pix_fmt yuv420p -strict -2 -filter:v "minterpolate=fps=25:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1" video.mp4
ffmpeg -y -i video.mp4 -i $aFILENAME  -c:v copy -c:a copy /content/video_waudio.mp4

In [12]:
# @title Download video
from google.colab import files
filepath="/content/video_waudio.mp4"#@param{type:'string'}
print(filepath)
files.download(filepath)

/content/video_waudio.mp4


FileNotFoundError: ignored

In [ ]:
# @title Install SRCNN deps
!cd /content/
!git clone https://github.com/Mirwaisse/SRCNN.git
!for x in 4 3 2 ;do  curl "https://raw.githubusercontent.com/chigozienri/SRCNN/master/models/model_${x}x.pth" -o "model_${x}x.pth" ; done
!pip install imageio-ffmpeg glob &> /dev/null

In [ ]:
# @title Increase Resolution

# import subprocess in case this cell is run without the above cells
import subprocess
import glob

# Set zoomed = True if this cell is run
zoomed = True
working_dir='/content/output'#@param {type:"string"}
rangeProcess=False#@param {type:"boolean"}
init_frame = 0#@param {type:"number"}
last_frame = 5001#@param {type:"number"}
zoomAmount = 4#@param {type:"number"}
image_path = f'{working_dir}/init-[0-9]*.png'
if rangeProcess:
  filesList=range(init_frame   , last_frame)
else:
  filesList=glob.glob(image_path).sort
for i in filesList: 
#for i in range(init_frame   , last_frame): #
    if rangeProcess:
        filename = f"init-{i:08}.png"
    else:
        filename = 'init-'+i.strip(working_dir)
    cmd = [
        'python',
        '/content/SRCNN/run.py',
        '--zoom_factor',
        str(zoomAmount),  # Note if you increase this, you also need to change the model.
        '--model',
        '/content/model_'+str(zoomAmount)+'x.pth',  # 2x, 3x and 4x are available from the repo above
        '--image',
        filename,
        '--cuda'
    ]
    print(f'Upscaling frame {i} : {working_dir}/{filename}')
    #working_dir='/content/drive/MyDrive/ColabNotebooks/steps'
    process = subprocess.Popen(cmd, cwd=f'{working_dir}')
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        print(
            "You may be able to avoid this error by backing up the frames,"
            "restarting the notebook, and running only the video synthesis cells,"
            "or by decreasing the resolution of the image generation steps. "
            "If you restart the notebook, you will have to define the `filepath` manually"
            "by adding `filepath = 'PATH_TO_THE_VIDEO'` to the beginning of this cell. "
            "If these steps do not work, please post the traceback in the github."
        )
        raise RuntimeError(stderr)


In [ ]:
# @title Make Increased Resolution Video
aFILENAME="/content/input/init.mp3"#@param{type:'string'}
%cd /content/output
!ffmpeg -y -r 60 -i zoomed_init-%08d.png -b:v 8M -c:v h264_nvenc -pix_fmt yuv420p -strict -2 -filter:v "minterpolate='mi_mode=mci:mc_mode=aobmc:vsbmc=1:fps=60'" zoomed_video.mp4
!ffmpeg -y -i zoomed_video.mp4 -i $aFILENAME  -c:v copy -c:a copy /content/zoomed_video_waudio.mp4

In [ ]:
# @title Download Increased Resolution Video
from google.colab import files
filepath="/content/zoomed_video_waudio.mp4"#@param{type:'string'}
print(filepath)
files.download(filepath)

In [ ]:
# @title Convert stereo audio to 360 4ch - ambisonics-3d-audio
#https://raw.githubusercontent.com/TheJoin95/ambisonics-3d-audio/master/init.py
!pip install pydub 
import glob
from pydub import AudioSegment
from pydub.generators import WhiteNoise
from math import *
from random import *
import sys
mp3filepath='/content/input/init.mp3'#@param{type:'string'}
#/usr/bin/ffmpeg
#/usr/bin/ffprobe
AudioSegment.converter = '/usr/bin/ffmpeg'#%shell which ffmpeg
AudioSegment.ffmpeg = '/usr/bin/ffmpeg'#%shell which ffmpeg
AudioSegment.ffprobe = '/usr/bin/ffprobe'#%shell which ffprobe

def calc_pan(index):
	return cos(radians(index))

#playlist_songs = [AudioSegment.from_mp3(mp3_file) for mp3_file in glob("mp3/*.mp3")]

#first_song = playlist_songs.pop(0)
interval = 0.2 * 1000 # sec
song = AudioSegment.from_mp3(mp3filepath)
song_inverted = song.invert_phase()
song.overlay(song_inverted)

splitted_song = splitted_song_inverted = []
song_start_point = 0

print("split song in part")
while song_start_point+interval < len(song):
    splitted_song.append(song[song_start_point:song_start_point+interval])
    song_start_point += interval

if song_start_point < len(song):
    splitted_song.append(song[song_start_point:])

print("end splitting")
print("total pieces: " + str(len(splitted_song)))

ambisonics_song = splitted_song.pop(0)
pan_index = 0
for piece in splitted_song:
    pan_index += 5
    piece = piece.pan(calc_pan(pan_index))
    ambisonics_song = ambisonics_song.append(piece, crossfade=interval/50)


# lets save it!
out_f = open(mp3filepath.strip('.mp3')+"_out.mp3", 'wb')
ambisonics_song.export(out_f, format='mp3')

In [ ]:
# @title Create zoomed 360 video for social media
audin='/content/input/init_out.mp3'#@param{type:'string'}
vidin='/content/output/zoomed_video.mp4'#@param{type:'string'}
vidout360='/content/360_video_waudio.mp4'#@param{type:'string'}
from pathlib import Path
%env vidin={vidin}
%env audin={audin}
base=Path(vidin).stem
outbase=Path(vidout360).stem
%env base={base}
%cd /content/input/ 
#!base=`echo ${vidin}|sed 's|\..*$||g'`
%env ffworkdir=/content/input/
!echo $vidin $audin $base $vidour360

!ffmpeg -y -i  {vidin} -c:v libx264 -b:v 40m -vf scale=3840:2160 -r 30 -profile main -pix_fmt yuv420p {base}.encodedforjump.video.360.mono.mp4
	   
!ffmpeg -y -i  {audin} -channel_layout 4.0 -c:a aac -b:a 128k -strict -2 {base}-ACN-SN3D-4ch-aac128.mp4
	   
!ffmpeg -y -i  {base}.encodedforjump.video.360.mono.mp4 -i {base}-ACN-SN3D-4ch-aac128.mp4 -channel_layout 4.0 -c:a copy -c:v copy -shortest {base}.encodedforjump.360.mono.mp4


%cd /content/
!pip install git+https://github.com/google/spatial-media.git
!git clone https://github.com/google/spatial-media.git
%cd spatial-media
!python spatialmedia -i --stereo=none --spatial-audio {base}.encodedforjump.360.mono.mp4 {vidout360}

In [ ]:
# @title Download 360 degree Video
from google.colab import files
filepath="/content/360_video_waudio.mp4"#@param{type:'string'}
print(filepath)
files.download(filepath)